In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## GET AVERAGE RETURNS OVER TIME OF THE AVAILABLE FUNDS

In [ ]:
# Connection to WRDS
import wrds
db = wrds.Connection(wrds_username='etiennebruno')
#db = wrds.Connection(wrds_username='zadaf')

In [ ]:
# Import CIKs from the dataframe of all cleaned filings
from utils import *
ciks = ', '.join(list(df_from_filings().cik.unique()))

In [ ]:
# Get all cik number available (intersection between our dataframe and the data on CRSP)
ciks_data = db.raw_sql(f'''
        select distinct comp_cik
        from crsp_q_mutualfunds.crsp_cik_map
        where comp_cik in ({ciks})
''')
cik_available = list(ciks_data.comp_cik)

cik_available_crsp_pkl = ['000'+str(int(c)) for c in cik_available]
save_pkl(cik_available_crsp_pkl, 'data/cik_available_crsp.pkl')

cik_available = ', '.join([str(i) for i in cik_available])

In [ ]:
# With the mapping table of CRSP, get all companies' fund number
crsp_fundno_available = db.raw_sql(f'''
        select *
        from crsp_q_mutualfunds.crsp_cik_map
        where comp_cik in ({cik_available})
''')
crsp_fundno_available

In [ ]:
# Convert the list of fund numbers to a list of string type for future SQL queries
list_float_fundnos = list(crsp_fundno_available.crsp_fundno.unique())
crsp_fundno_available_list_str = ', '.join([str(i) for i in list_float_fundnos])

In [ ]:
# Get all available returns for each fund
monthly_returns_all_funds = db.raw_sql(f'''
        select *
        from crsp_q_mutualfunds.monthly_returns
        where crsp_fundno in ({crsp_fundno_available_list_str})
        order by caldt desc
''')
# monthly_returns_all_funds.to_pickle('monthly_returns_all_funds.pkl')

In [ ]:
crsp_fundno_available[crsp_fundno_available.comp_cik == 1593547]

In [ ]:
funds_returns = pd.merge(monthly_returns_all_funds, crsp_fundno_available)[['caldt', 'comp_cik','mret']]
funds_returns['comp_cik'] = funds_returns.comp_cik.apply(lambda cik: '000'+str(int(cik)))
funds_returns.to_pickle('data/funds_returns.pkl')

In [ ]:
monthly_returns_all_funds = monthly_returns_all_funds[monthly_returns_all_funds['mret']<4]

In [ ]:
monthly_returns_all_funds[monthly_returns_all_funds['mret']>=1]

## GET AVERAGE RETURNS OVER TIME OF THE AVAILABLE FUNDS - GROUPED BY MONTH

In [ ]:
# Create a new dataframe with some statistic and the average return
monthly_returns_all_funds_aggregated = monthly_returns_all_funds[['caldt', 'mret']].groupby('caldt').agg(
    count=('mret', 'count'),
    mret=('mret', 'mean'),
    mret_max=('mret', 'max'),
    mret_min=('mret', 'min'),
    mret_sdt=('mret', 'std'),
)

In [ ]:
monthly_returns_all_funds_aggregated.reset_index(inplace=True)
monthly_returns_all_funds_aggregated['caldt'] = pd.to_datetime(monthly_returns_all_funds_aggregated['caldt'], format= '%Y-%m-%d')

In [ ]:
monthly_returns_all_funds_aggregated = monthly_returns_all_funds_aggregated.loc[monthly_returns_all_funds_aggregated['caldt'] >= '2000-01-01']
monthly_returns_all_funds_aggregated

## GET RETURN OF THE MARKET

In [ ]:
index_return_query = db.raw_sql("""
    select  
    date,
    vwretd 
    from crsp.msi
    where 
    date>='2000-01-01'""", date_cols=['date']
    )

In [ ]:
left = monthly_returns_all_funds_aggregated[['mret', 'caldt', 'count']]
right = index_return_query[['date', 'vwretd']]
cumulative_data = pd.merge(left=left, right=right, how='inner', left_on='caldt', right_on='date')

# Compute cumulative returns for both avg of funds and market
cumulative_data['cum_ret_funds'] = (1 + cumulative_data['mret']).cumprod() - 1
cumulative_data['cum_ret_mkt'] = (1 + cumulative_data['vwretd']).cumprod() - 1

In [ ]:
index_return_query[['date', 'vwretd']].to_pickle('market_returns.pkl')

In [ ]:
index_return_query

### Get Risk Free Rate

In [ ]:
risk_free =db.raw_sql("""select mcaldt, tmytm
                                from crsp.tfz_mth_rf
                                where kytreasnox = 2000001
                                    and mcaldt>='2000-01-01'
                                    --and mcaldt<='2021-12-31'
                                """, date_cols=['mcaldt'])

In [ ]:
risk_free.mcaldt.apply(lambda date: date.year)
risk_free['year'] = risk_free.mcaldt.dt.year
risk_free = risk_free.groupby('year').last().reset_index()
risk_free.to_pickle('risk_free.pkl')

## GRAPH OF AVERAGE RETURNS OVER TIME WITH BENCHMARKS

In [ ]:
# Rename df for better modularity
data = monthly_returns_all_funds_aggregated.copy()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style="darkgrid")
sns.set_style("whitegrid", {'axes.grid' : False})
sns.set(rc={"figure.figsize":(18, 8)})
#plt.rcParams['xtick.labelsize'] = 'medium'
#plt.rcParams['ytick.labelsize'] = 'medium'

In [ ]:
plt.tick_params(left=True,
                bottom=True,
                labelleft=True,
                labelbottom=True)

# Plot the average return of all mutual funds
X=data['caldt'].dt.strftime('%Y-%m-%d')
Y=data['mret']
sns.lineplot(x=X, y=Y, linestyle='--')

# Add variance
lower_bound = data['mret'] + data['mret_sdt']
upper_bound = data['mret'] - data['mret_sdt']

plt.fill_between(X, lower_bound, upper_bound, alpha=.3)

# Average Value Weighted Index - market
X=index_return_query['date'].dt.strftime('%Y-%m-%d')
Y=index_return_query['vwretd']
sns.lineplot(x=X, y=Y, linestyle='--')

plt.xlabel('Time')
plt.ylabel('Average Monthly Return')
plt.xticks(X[::12], rotation=45)

plt.legend(['average monthly return of mutual funds', 'variance of the funds\' returns', 'value weighted index of the market'])
#plt.legend(['value weighted index of the market'])
#plt.savefig('data/graphs/annual_return_market_plots.png', dpi=200)
#plt.savefig('data/graphs/annual_return_plots.png', dpi=200)
#plt.savefig('data/graphs/annual_return_plots_with_market.png', dpi=200)
plt.show()

In [ ]:
import statsmodels.api as sm

endog = data['mret']
exog = sm.add_constant(data['vwretd'], prepend=False)
# Fit and summarize OLS model
mod = sm.OLS(endog, exog)
res = mod.fit()
print(res.summary())
#print(res.summary().as_latex())

In [ ]:
from scipy import stats

# Correlation betwwen the cumulatives returns of the market and of the funds
Y=data['mret']
Z=data['vwretd']
print(stats.pearsonr(Y, Z))

### GET AND PLOT CUMULATIVE RETURNS

In [ ]:
# Rename df for better modularity
data = cumulative_data.copy()

In [ ]:
fig = plt.figure(figsize=(15, 15))
ax1 = plt.subplot(2, 1, 1)

# Plot the cumulative return of all mutual funds
X=data['caldt'].dt.strftime('%Y-%m-%d')
Y=data['cum_ret_funds']*100
sns.lineplot(x=X, y=Y, linestyle='--', ax=ax1)

# Cumulative Value Weighted index return - market
X=data['date'].dt.strftime('%Y-%m-%d')
Y=data['cum_ret_mkt']*100
sns.lineplot(x=X, y=Y, linestyle='--', ax=ax1)

# Setting for both x-axes
plt.xlabel('')
plt.ylabel('Cumulative return (in %)')
plt.title('Cumulative Return of the average return of the mutual funds and the value weighted market index')
plt.xticks(X[::12], rotation=45)

# Add number of funds per date
ax2 = plt.subplot(2, 1, 2)
X=data['caldt'].dt.strftime('%Y-%m-%d')
ax2.grid(False)
sns.lineplot(x=X, y='count', data=data, ax=ax2)
lower_bound = [0]*len(data['caldt'])
upper_bound = data['count']
plt.fill_between(X, lower_bound, upper_bound, alpha=.3)
ax2.set_ylim([0,175])

# Setting for both x-axes
plt.xlabel('Time')
plt.ylabel('Number of funds')
plt.title('Number of mutual funds taken into consideration at each time t')
plt.xticks(X[::12], rotation=45)


#plt.legend(['Average Performance of mutual funds', 'Value Weighted Market Index'])
#plt.savefig('data/graphs/comparisons_with_vw_market_two_figures.png', dpi=200)
# set the spacing between subplots
#plt.subplots_adjust(hspace=0.4)
fig.tight_layout()
plt.show()

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

# Plot the cumulative return of all mutual funds
X=data['caldt'].dt.strftime('%Y-%m-%d')
Y=data['cum_ret_funds']*100
sns.lineplot(x=X, y=Y, linestyle='--', ax=ax1)

# Cumulative Value Weighted index return - market
X=data['date'].dt.strftime('%Y-%m-%d')
Y=data['cum_ret_mkt']*100
sns.lineplot(x=X, y=Y, linestyle='--', ax=ax1)

# Setting for both x-axes
plt.xlabel('Time')
plt.ylabel('Cumulative return (in %)')
plt.title('Cumulative Return of the average return of the mutual funds and the value weighted market index')
plt.legend(['Average Performance of mutual funds', 'Value Weighted Market Index'])
plt.xticks(X[::12], rotation=45)

# Add number of funds per date
ax2 = ax1.twinx()
X=data['caldt'].dt.strftime('%Y-%m-%d')
ax2.grid(False)
sns.lineplot(x=X, y='count', data=data, ax=ax2, linestyle=':')
#lower_bound = [0]*len(data['caldt'])
#upper_bound = data['count']
#plt.fill_between(X, lower_bound, upper_bound, alpha=.3)
ax2.set_ylim([0,200])


plt.legend(['Number of funds taken into consideration at each time t'])
#plt.savefig('data/graphs/comparisons_with_vw_market_one_figure.png', dpi=200)
plt.show()

In [ ]:
# Plot the cumulative return of all mutual funds
X=data['caldt'].dt.strftime('%Y-%m-%d')
Y=data['cum_ret_funds']*100
sns.lineplot(x=X, y=Y, linestyle='--')

# Cumulative Value Weighted index return - market
X=data['date'].dt.strftime('%Y-%m-%d')
Y=data['cum_ret_mkt']*100
sns.lineplot(x=X, y=Y, linestyle='--')

# Setting for both x-axes
plt.xlabel('')
plt.ylabel('Cumulative return (in %)')
plt.title('Cumulative Return of the average return of the mutual funds and the value weighted market index')
plt.xticks(X[::12], rotation=45)

# Setting for both x-axes
plt.xlabel('Time')
plt.ylabel('Number of funds')
plt.title('Cumulative returns of mutual funds and of the market')
plt.xticks(X[::12], rotation=45)


plt.legend(['Cumulative return of the average monthly returns of mutuals funds', 'Cumulative return of the value weighted market index'])
#plt.savefig('data/graphs/cumulative_returns.png', dpi=200)
fig.tight_layout()
plt.show()

In [ ]:
import statsmodels.api as sm

endog = data['cum_ret_funds']
exog = sm.add_constant(data['cum_ret_mkt'], prepend=False)
# Fit and summarize OLS model
mod = sm.OLS(endog, exog)
res = mod.fit()
print(res.summary())
#print(res.summary().as_latex())

In [ ]:
# Correlation betwwen the cumulatives returns of the market and of the funds
Y=data['cum_ret_funds']
Z=data['cum_ret_mkt']
print(stats.pearsonr(Y, Z))

In [ ]:
# Add number of funds per date
X=data['caldt'].dt.strftime('%Y-%m-%d')
ax2.grid(False)
sns.lineplot(x=X, y='count', data=data, ax=ax2)
lower_bound = [0]*len(data['caldt'])
upper_bound = data['count']
plt.fill_between(X, lower_bound, upper_bound, alpha=.3)
ax2.set_ylim([0,175])

# Setting for both x-axes
plt.xlabel('Time')
plt.ylabel('Number of funds')
plt.title('Number of mutual funds taken into consideration at each time t')
plt.xticks(X[::12], rotation=45)

#plt.savefig('data/graphs/number_of_funds_evoluation.png', dpi=200)
fig.tight_layout()
plt.show()

## FIND VARIABLE NAME IN SCHEMA

In [ ]:
%%time
for table in db.list_tables(library='crsp'):
    try:
        l= list(filter(lambda x: ('crsp_fundno' in x) and ('cik' in x), db.get_table(library='crsp_q_mutualfunds', table=table, obs=1).keys()))
    except:
        pass
    if len(l) > 0:
        print(table, "---", l)

In [ ]:
%%time
for table in db.list_tables(library='crsp'):
    try:
        tab_keys = db.get_table(library='crsp_q_mutualfunds', table=table, obs=1).keys()
        l = list(filter(lambda x: 'cik' in x, tab_keys))
    except:
        pass
    if len(l) > 0:
        print(table, "---", l, tab_keys)

In [ ]:
%%time
for table in db.list_tables(library='crsp'):
    try:
        l= list(filter(lambda x: 'cik' in x, db.get_table(library='crsp', table=table, obs=1).keys()))
    except:
        pass
    if len(l) > 0:
        print(table, "---", l)

## TRY SOME QUERIES

In [ ]:
# Connection to WRDS
import wrds
db = wrds.Connection(wrds_username='etiennebruno')

In [ ]:
db.list_libraries().sort()
db.list_libraries()

In [ ]:
# List the tables of a given library
db.list_tables(library='crsp')

In [ ]:
# Get data
daily = db.get_table(library='crsp', table='fund_summary2', obs=10)

In [ ]:
# Get funds name over a given period
df_funds_map = db.raw_sql('''
        select distinct on (fund_name) fund_name, ticker, ncusip, cusip8
        from crsp.portnomap
        where begdt <= '2020-01-01' --and enddt >= '2022-01-01'
        order by fund_name asc
''')

In [ ]:
# Get funds name over containing some strings of the S&P 500
db.raw_sql('''
        select distinct fund_name
        from crsp.portnomap
        where lower(fund_name::text) LIKE '%%s&p%%'
            and lower(fund_name::text) LIKE '%%500%%'
            --and lower(fund_name::text) LIKE '%%bond%%'
            --and lower(fund_name::text) LIKE '%%national%%'
            --and caldt >= '2010-01-31'
     limit 100
''')

In [ ]:
db.raw_sql('''
        select *
        from crsp.fund_summary2
        where fund_name LIKE '%%FundVantage Trust: Gotham Enhanced S&P 500 Index Fund; Institutional Class Shares%%'
        --and caldt >= '2010-01-31'
''')

In [ ]:
db.raw_sql('''
        select *
        from crsp.portnomap
        where (fund_name::text) LIKE '%%FundVantage Trust: Gotham Enhanced S&P 500 Index Fund; Institutional Class Shares%%'
        --and caldt >= '2010-01-31'
''')

In [ ]:
db.raw_sql('''
        select crsp_fundno, count(mret), max(mret), avg(mret)
        from crsp_q_mutualfunds.monthly_tna_ret_nav
        group by crsp_fundno
        order by count(mret) desc
     limit 10
''')